# Adventureworks Data Warehouse ETL - Project 1 (SQL) 
**Goal:** build dimensional data warehouse in My SQL from the Adventureworks OLTP schema using Python and SQLAlchemy

**Steps:**
1. Create the data warehouse schema
2. extract the data from adventureworks views
3. transform (cleaning and surrogate keys)
4. load to the data warehouse tables
5. build date dimension
6. load fact tables
7. validate with SQL queries

#### Import the Necessary Libraries

In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime

#### Imports + Connection Variables 

In [2]:
# MySQL server & databases
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "shep1234"

src_dbname = "adventureworks"
dst_dbname = "adventureworks_dw"

#### Define helper functions

In [3]:
## MySQL helpers
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create DW Schema or Database
##### Only run once

In [ ]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()


#### Create & Populate Dimension Tables 

##### Extract data from source

In [4]:
## customers 
sql_customers = "SELECT * FROM adventureworks.dim_customers_vw;"
df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
df_customers.head(2)

,CustomerID,AccountNumber,CustomerType,AddressType,AddressLine1,AddressLine2,City,StateProvinceCode,State_Province,IsOnlyStateProvinceFlag,PostalCode,CountryRegionCode,Country_Region,Sales Territory Group,Sales Territory
0,1,AW00000001,S,Main Office,2251 Elliot Avenue,None,Seattle,WA,Washington,b'\x00',98104,US,United States,North America,Northwest
1,2,AW00000002,S,Shipping,7943 Walnut Ave,None,Renton,WA,Washington,b'\x00',98055,US,United States,North America,Northwest


In [5]:
## employees
sql_employees = "SELECT * FROM adventureworks.dim_employee_vw;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
df_employees.head(2)

,EmployeeID,NationalIDNumber,LoginID,ManagerID,FirstName,MiddleName,LastName,Title,EmailAddress,EmailPromotion,Phone,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag
0,1,14417807,adventure-works\guy1,16.0,Guy,R,Gilbert,Production Technician - WC60,guy1@adventure-works.com,0,320-555-0195,1972-05-15,M,M,1996-07-31,b'\x00',21,30,b'\x01'
1,2,253022876,adventure-works\kevin0,6.0,Kevin,F,Brown,Marketing Assistant,kevin0@adventure-works.com,2,150-555-0189,1977-06-03,S,M,1997-02-26,b'\x00',42,41,b'\x01'


In [6]:
## products 
sql_products = "SELECT * FROM adventureworks.dim_products_vw;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,DaysToManufacture,ProductLine,Class,Style,ProductCategory,ProductSubcategory,ProductModel,SellStartDate,SellEndDate,DiscontinuedDate
0,1,Adjustable Race,AR-5381,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,0,None,None,None,None,None,None,1998-06-01,NaT,None
1,2,Bearing Ball,BA-8327,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,0,None,None,None,None,None,None,1998-06-01,NaT,None


In [7]:
## vendors 
sql_vendors = "SELECT * FROM adventureworks.dim_vendors_vw;"
df_vendors = get_dataframe(user_id, pwd, host_name, src_dbname, sql_vendors)
df_vendors.head(2)

,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,AddressType,AddressLine1,AddressLine2,City,StateProvinceCode,State_Province,PostalCode
0,1,INTERNAT0001,International,1,b'\x01',b'\x01',Main Office,683 Larch Ct.,None,Salt Lake City,UT,Utah,84101
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,b'\x01',b'\x01',Main Office,8547 Catherine Way,None,Tacoma,WA,Washington,98403


##### Transform  

In [8]:
## customers
drop_cols = ['AddressLine2','StateProvinceCode', 'IsOnlyStateProvinceFlag', 'CountryRegionCode', 'Sales Territory Group']
df_customers.drop(drop_cols, axis=1, inplace=True)
df_customers.rename(columns={"CustomerID":"customer_id"}, inplace=True)
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))
df_customers.head(2)

,customer_key,customer_id,AccountNumber,CustomerType,AddressType,AddressLine1,City,State_Province,PostalCode,Country_Region,Sales Territory
0,1,1,AW00000001,S,Main Office,2251 Elliot Avenue,Seattle,Washington,98104,United States,Northwest
1,2,2,AW00000002,S,Shipping,7943 Walnut Ave,Renton,Washington,98055,United States,Northwest


In [9]:
## employees
drop_cols = ['MiddleName', 'EmailPromotion','SalariedFlag','CurrentFlag']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees.rename(columns={"EmployeeID":"employee_id"}, inplace=True)
df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))
df_employees.head(2)

,employee_key,employee_id,NationalIDNumber,LoginID,ManagerID,FirstName,LastName,Title,EmailAddress,Phone,BirthDate,MaritalStatus,Gender,HireDate,VacationHours,SickLeaveHours
0,1,1,14417807,adventure-works\guy1,16.0,Guy,Gilbert,Production Technician - WC60,guy1@adventure-works.com,320-555-0195,1972-05-15,M,M,1996-07-31,21,30
1,2,2,253022876,adventure-works\kevin0,6.0,Kevin,Brown,Marketing Assistant,kevin0@adventure-works.com,150-555-0189,1977-06-03,S,M,1997-02-26,42,41


In [10]:
## products
drop_cols = ['MakeFlag','FinishedGoodsFlag', 'Color','ProductLine','Class', 'Style', 'ProductCategory', 'ProductModel', 'SellEndDate', 'DiscontinuedDate']
df_products.drop(drop_cols, axis=1, inplace=True)
df_products.rename(columns={"ProductID":"product_id"}, inplace=True)
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head(2)

,product_key,product_id,Name,ProductNumber,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductSubcategory,SellStartDate
0,1,1,Adjustable Race,AR-5381,1000,750,0.0,0.0,None,None,None,NaN,0,None,1998-06-01
1,2,2,Bearing Ball,BA-8327,1000,750,0.0,0.0,None,None,None,NaN,0,None,1998-06-01


In [11]:
## vendors 
drop_cols = ['PreferredVendorStatus', 'ActiveFlag', 'AddressLine2', 'StateProvinceCode']
df_vendors.drop(drop_cols, axis=1, inplace=True)
df_vendors.rename(columns={"VendorID":"vendor_id"}, inplace=True)
df_vendors.insert(0, "vendor_key", range(1, df_vendors.shape[0]+1))
df_vendors.head(2)

,vendor_key,vendor_id,AccountNumber,Name,CreditRating,AddressType,AddressLine1,City,State_Province,PostalCode
0,1,1,INTERNAT0001,International,1,Main Office,683 Larch Ct.,Salt Lake City,Utah,84101
1,2,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,Main Office,8547 Catherine Way,Tacoma,Washington,98403


In [12]:
def bit_to_int(x):
    return 1 if x in (1, True, b"\x01") else 0

for col in ["SalariedFlag","CurrentFlag"]:
    if col in df_employees.columns:
        df_employees[col] = df_employees[col].apply(bit_to_int)

##### Load transformed daraframes into new data warehouse (create new tables) 

In [13]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_employees', df_employees, 'employee_key'),
          ('dim_products', df_products, 'product_key'),
          ('dim_vendors', df_vendors, 'vendor_key')]

In [14]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### Create & Populate dim_date

In [15]:
sql_dates = """
SELECT DISTINCT DATE(OrderDate) AS d FROM salesorderheader
UNION SELECT DISTINCT DATE(ShipDate) FROM salesorderheader
UNION SELECT DISTINCT DATE(DueDate)  FROM salesorderheader
UNION SELECT DISTINCT DATE(OrderDate) FROM purchaseorderheader
UNION SELECT DISTINCT DATE(ShipDate)  FROM purchaseorderheader;
"""
df_date = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dates)


In [16]:
# build date key = YYYYMMDD and order columns 
df_date["full_date"] = pd.to_datetime(df_date["d"], errors="coerce").dt.date
df_date = df_date.dropna(subset=["full_date"]).copy()
df_date["date_key"] = (
    pd.to_datetime(df_date["full_date"]).dt.year * 10000
    + pd.to_datetime(df_date["full_date"]).dt.month * 100
    + pd.to_datetime(df_date["full_date"]).dt.day
)
df_date = df_date[["date_key","full_date"]].drop_duplicates()

In [17]:
# load to DW
set_dataframe(user_id, pwd, host_name, dst_dbname, df_date, "dim_date", "date_key", "insert")

#### Build Fact Tables

##### Extract flattened fact views from OLTP 

In [18]:
sql_fact_sales  = "SELECT * FROM fact_sales_orders_vw;"
df_fact_sales   = get_dataframe(user_id, pwd, host_name, src_dbname, sql_fact_sales)

In [19]:
sql_fact_purch  = "SELECT * FROM fact_purchase_orders_vw;"
df_fact_purch   = get_dataframe(user_id, pwd, host_name, src_dbname, sql_fact_purch)

##### prep ket look up DF from new DW dims

In [20]:
dim_dates   = get_dataframe(user_id, pwd, host_name, dst_dbname, "SELECT date_key, full_date FROM dim_date;")
dim_dates["full_date"] = pd.to_datetime(dim_dates["full_date"]).dt.date

dim_cust    = get_dataframe(user_id, pwd, host_name, dst_dbname, "SELECT customer_key, customer_id FROM dim_customers;")
dim_emp     = get_dataframe(user_id, pwd, host_name, dst_dbname, "SELECT employee_key, employee_id FROM dim_employees;")
dim_prod    = get_dataframe(user_id, pwd, host_name, dst_dbname, "SELECT product_key, product_id FROM dim_products;")
dim_vendor  = get_dataframe(user_id, pwd, host_name, dst_dbname, "SELECT vendor_key, vendor_id FROM dim_vendors;")

##### Sales fact: date keys attached and dim surrogate keys

In [21]:
# Date keys
for col in ["OrderDate","ShipDate","DueDate"]:
    if col in df_fact_sales.columns:
        df_fact_sales[col] = pd.to_datetime(df_fact_sales[col], errors="coerce").dt.date
        join_df = dim_dates.rename(columns={"date_key": f"{col.lower()}_key", "full_date": col})
        df_fact_sales = df_fact_sales.merge(join_df, how="left", on=col).drop(columns=[col])

In [22]:
# Entity keys 
if "CustomerID" in df_fact_sales.columns:
    df_fact_sales = df_fact_sales.merge(dim_cust, how="left",
                                        left_on="CustomerID", right_on="customer_id") \
                                 .drop(columns=["CustomerID","customer_id"])
if "SalesPersonID" in df_fact_sales.columns:
    df_fact_sales = df_fact_sales.merge(dim_emp, how="left",
                                        left_on="SalesPersonID", right_on="employee_id") \
                                 .drop(columns=["SalesPersonID","employee_id"])
if "ProductID" in df_fact_sales.columns:
    df_fact_sales = df_fact_sales.merge(dim_prod, how="left",
                                        left_on="ProductID", right_on="product_id") \
                                 .drop(columns=["ProductID","product_id"])


In [23]:
df_fact_sales.insert(0, "fact_sales_key", range(1, len(df_fact_sales)+1))
set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_sales,
              "fact_sales_orders", "fact_sales_key", "insert")

##### Purchase fact

In [24]:
for col in ["OrderDate", "ShipDate", "DueDate"]:
    if col in df_fact_purch.columns:
        df_fact_purch[col] = pd.to_datetime(df_fact_purch[col], errors="coerce").dt.date
        join_df = dim_dates.rename(columns={"date_key": f"{col.lower()}_key", "full_date": col})
        df_fact_purch = df_fact_purch.merge(join_df, how="left", on=col).drop(columns=[col])

if "EmployeeID" in df_fact_purch.columns:
    df_fact_purch = df_fact_purch.merge(dim_emp, how="left",
                                        left_on="EmployeeID", right_on="employee_id") \
                                 .drop(columns=["employee_id"])
if "VendorID" in df_fact_purch.columns:
    df_fact_purch = df_fact_purch.merge(dim_vendor, how="left",
                                        left_on="VendorID", right_on="vendor_id") \
                                 .drop(columns=["vendor_id"])
if "ProductID" in df_fact_purch.columns:
    df_fact_purch = df_fact_purch.merge(dim_prod, how="left",
                                        left_on="ProductID", right_on="product_id") \
                                 .drop(columns=["product_id"])

df_fact_purch.insert(0, "fact_purchase_key", range(1, len(df_fact_purch)+1))
set_dataframe(user_id, pwd, host_name, dst_dbname,
              df_fact_purch, "fact_purchase_orders", "fact_purchase_key", "insert")


#### Quick Validate

In [25]:
def validate(table, key):
    sql = f"""
      SELECT COUNT(*) AS row_count,
             MIN({key}) AS min_key,
             MAX({key}) AS max_key
      FROM {table}
    """
    return get_dataframe(user_id, pwd, host_name, dst_dbname, sql)

print("Sales fact:")
display(validate("fact_sales_orders", "fact_sales_key"))

print("Purchase fact:")
display(validate("fact_purchase_orders", "fact_purchase_key"))

print("Dims:")
for t,k in [("dim_customers","customer_key"),("dim_employees","employee_key"),
            ("dim_products","product_key"),("dim_vendors","vendor_key"),("dim_date","date_key")]:
    display(validate(t,k))


Sales fact:


,row_count,min_key,max_key
0,122579,1,122579


Purchase fact:


,row_count,min_key,max_key
0,96668,1,96668


Dims:


,row_count,min_key,max_key
0,19220,1,19220


,row_count,min_key,max_key
0,290,1,290


,row_count,min_key,max_key
0,504,1,504


,row_count,min_key,max_key
0,104,1,104


,row_count,min_key,max_key
0,1174,20010517,20040912


##### Fact sales orders has 122,579 rows. Fact Purchase Orders has 87,880 rows 
##### Dimensions Rows: 
1. customers: 19,220
2. employees: 290
3. products: 504
4. vendors: 104
5. date: 1,174 (~3 years) 


### SQL DEMO QUERIES 

In [26]:
# 1. Total sales and quantity by customer and sales territory
sql_demo_1 = """
SELECT
  dc.AccountNumber,
  dc.`Sales Territory` AS sales_territory,
  ROUND(SUM(f.LineTotal), 2) AS total_sales,
  SUM(f.OrderQty)            AS total_quantity
FROM fact_sales_orders f
JOIN dim_customers dc ON f.customer_key = dc.customer_key
GROUP BY dc.AccountNumber, dc.`Sales Territory`
ORDER BY total_sales DESC;
"""
get_dataframe(user_id, pwd, host_name, dst_dbname, sql_demo_1)


,AccountNumber,sales_territory,total_sales,total_quantity
0,AW00000004,Southwest,1173049.89,1960.0
1,AW00000254,Northwest,886632.23,3212.0
2,AW00000697,Northwest,877107.19,1558.0
3,AW00000278,Southeast,863983.10,712.0
4,AW00000170,Southeast,853849.18,1322.0
...,...,...,...,...
19114,AW00028016,Southwest,2.29,1.0
19115,AW00028093,Northwest,2.29,1.0
19116,AW00028968,Southwest,2.29,1.0
19117,AW00028781,Southwest,2.29,1.0


In [27]:
sql_demo_2 = """
SELECT
  dv.Name AS vendor_name,
  YEAR(dd.full_date) AS year,
  QUARTER(dd.full_date) AS quarter,
  ROUND(SUM(f.TotalDue), 2) AS total_spend
FROM fact_purchase_orders f
JOIN dim_vendors dv ON f.vendor_key = dv.vendor_key
JOIN dim_date dd    ON f.orderdate_key = dd.date_key
GROUP BY dv.Name, year, quarter
ORDER BY total_spend DESC;
"""
get_dataframe(user_id, pwd, host_name, dst_dbname, sql_demo_2)


,vendor_name,year,quarter,total_spend
0,Chicago City Saddles,2004,3,42047792.41
1,Chicago City Saddles,2004,2,40043160.03
2,Chicago City Saddles,2004,1,38063587.31
3,Professional Athletic Consultants,2004,2,29882345.51
4,Superior Bicycles,2004,2,28796005.75
...,...,...,...,...
755,Northern Bike Travel,2002,1,450.65
756,Northern Bike Travel,2002,2,450.65
757,Northern Bike Travel,2002,3,450.65
758,Northern Bike Travel,2003,1,450.65
